# Importing Gaia data to Firefly

*Building off of the tutorial here: http://gea.esac.esa.int/archive-help/index.html *

*The main table is gaiadr2.gaia_source, and [here](http://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html) is the description of the columns*

In [1]:
%load_ext autoreload
%autoreload 2
from FIREreader import FIREreader

/Users/ageller/anaconda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import astropy.units as u
from astropy.coordinates import ICRS
from astropy.coordinates.sky_coordinate import SkyCoord
from astroquery.gaia import Gaia
import numpy as np

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


*Do the following to load and look at the available Gaia table names:*

In [ ]:
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print (table.get_qualified_name())

In [ ]:
gaiadr2_table = Gaia.load_table('gaiadr2.gaia_source')
for column in (gaiadr2_table.get_columns()):
    print(column.get_name())

### Next, we retrieve the data 

*Note: The query to the archive is with ADQL (Astronomical Data Query Language). *

In [74]:
#NOTE: not all sources have RVs
#cmd = "SELECT TOP 100 ra, dec, parallax, pmra, pmdec, radial_velocity FROM gaiadr2.gaia_source"

#NOTE: It looks like there are 480235 sources with g <=10 and with parallaxes
N = 2e6
cmd = "SELECT TOP " + str(np.int(N))+ " ra, dec, parallax, pmra, pmdec, radial_velocity FROM gaiadr2.gaia_source \
WHERE phot_g_mean_mag<=10 \
AND parallax IS NOT NULL "#\
#AND radial_velocity IS NOT NULL"
print(cmd)

#synchronous commands are OK for jobs with < 2000 output rows
#job = Gaia.launch_job(cmd, dump_to_file=False) 

#asynchronous commands are giving me an error!
job = Gaia.launch_job_async(cmd, dump_to_file=False)

print (job)

SELECT TOP 2000000 ra, dec, parallax, pmra, pmdec, radial_velocity FROM gaiadr2.gaia_source WHERE phot_g_mean_mag<=10 AND parallax IS NOT NULL 


Query finished.
<Table masked=True length=480235>
      name       dtype     unit    format                description                 n_bad 
--------------- ------- ---------- ------ ------------------------------------------ ------
             ra float64        deg {!r:>}                            Right ascension      0
            dec float64        deg {!r:>}                                Declination      0
       parallax float64        mas {!r:>}                                   Parallax      0
           pmra float64 mas.yr**-1 {!r:>} Proper motion in right ascension direction      0
          pmdec float64 mas.yr**-1 {!r:>}     Proper motion in declination direction      0
radial_velocity float64   km.s**-1 {!r:>}                            Radial velocity 141769
Jobid: 1525274887697O
Phase: COMPLETED
Owner: None
Output file: async_20180502102807.vot
Results: None


*Inspect the output table and number of rows:*

In [75]:
r = job.get_results()
print(len(r))
print(r)

480235
        ra                 dec         ...  radial_velocity  
       deg                 deg         ...      km.s**-1     
------------------ ------------------- ... ------------------
 274.4061143187362  -36.76240607839242 ...                 --
109.28559761663782 -37.097446909729776 ...                 --
311.55470104855004   33.97165785643453 ...                 --
 296.5649839440163  10.613253300439279 ...                 --
236.06755715935202   6.425826685796823 ...                 --
226.01721301839848 -25.282153195983703 ...                 --
 275.2486775206737  -29.82821901708563 ...                 --
 59.50763338363228 -13.508997297697881 ... 59.954691238798745
 261.3248898046725  -55.52999303063296 ...                 --
155.58177692208324    41.4996586971537 ...                 --
               ...                 ... ...                ...
 257.6789148577721  16.558220194211984 ... 19.998599315535344
 56.22345202413663  16.863282853375424 ...  61.72266799473432
 

### Convert these ra, dec, parallax coordinates to 3D cartesian

In [76]:
#it is odd that some of the parallaxes are negative!
xx = np.where(r['parallax'] < 0)[0]
print("Number where parallax < 0 :", len(xx))
dist = np.abs((r['parallax']).to(u.parsec, equivalencies=u.parallax()))

#this only contains the position, and in cartesian it is much faster
#astroC = SkyCoord(ra=r['ra'], dec=r['dec'], distance=dist).cartesian #transform_to(coord.Galactocentric)

#this would account for velocities also
pmra = np.array([x for x in r['pmra']]) * u.mas/u.yr
pmdec = np.array([x for x in r['pmdec']]) * u.mas/u.yr
rv = np.array([x for x in r['radial_velocity']]) * u.km/u.s 
xx = np.where(np.isnan(r['radial_velocity']) )[0]
print("Number without RVs :", len(xx))
#################
#################
# what should we about missing RV data?
#################
#################
for i in xx:
    rv[i] = 0*u.km/u.s

#NOTE: without RVs, the velocities are not correct
astroC = ICRS(ra=r['ra'], dec=r['dec'], distance=dist, pm_ra_cosdec=pmra, pm_dec=pmdec, radial_velocity=rv)

print(astroC.cartesian)
print(astroC.velocity)

Number where parallax < 0 : 1283


/Users/ageller/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':


Number without RVs : 141769
[(   3.1315411 ,  -40.64130966,  -30.45201359),
 ( -69.37069534,  198.25135221, -158.83565473),
 (  12.74090792,  -14.37330244,   12.94170654), ...,
 ( -42.86022242, -292.53338585, -449.35559793),
 (-592.33908293,  434.84088038,  -17.80786658),
 (  40.70484432,  250.62927038,   49.82438054)] pc
[(-33.55327794, 21.29293495, -31.86804804),
 ( 16.67902548,  7.65774824,   2.27356602),
 ( 17.50045152, 40.82817405,  28.11561606), ...,
 ( 13.58923521, 19.68270006, -14.10972638),
 ( -2.68062977, 10.99605326,  16.01736404),
 (  9.49774092, 37.85362816, -12.99971284)] km / s


*Now put these in the format expected by FIREreader, and convert to cartesian*

In [77]:
p = astroC.cartesian
v = astroC.velocity
coordinates = np.dstack((p.x.to(u.pc).value, p.y.to(u.pc).value, p.z.to(u.pc).value)).squeeze()
velocities = np.dstack((v.d_x.to(u.km/u.s).value, v.d_y.to(u.km/u.s).value, v.d_z.to(u.km/u.s).value)).squeeze()

 
print(coordinates[0:10])
print(velocities[0:10])

[[    3.1315411    -40.64130966   -30.45201359]
 [  -69.37069534   198.25135221  -158.83565473]
 [   12.74090792   -14.37330244    12.94170654]
 [  160.33843567  -320.67781207    67.18262268]
 [  -14.08974221   -20.94212353     2.84272035]
 [  -79.24365749   -82.10856      -53.89665179]
 [  108.7320149  -1183.62180629  -681.49846896]
 [   26.02758967    44.19954352   -12.32302709]
 [  -31.68304766  -207.65290788  -305.97646536]
 [  -69.26527358    31.44668857    67.29982114]]
[[ -33.55327794   21.29293495  -31.86804804]
 [  16.67902548    7.65774824    2.27356602]
 [  17.50045152   40.82817405   28.11561606]
 [  21.02644327   10.37017869   -0.68262904]
 [  14.19712109   -8.7897444     5.61358882]
 [ -25.83807412   38.62141367  -20.84814415]
 [ 230.46615975   88.51491928 -116.96158151]
 [  13.57355724   52.01586083  -41.42106981]
 [  -7.53879845   32.9227632   -21.56262377]
 [  30.10615173   35.05319079   14.60633989]]


### Create the Firefly data files using FIREreader

In [79]:
pname = 'Gaia'

reader = FIREreader()

#set all of these manually
reader.returnParts = [pname] 
reader.names = {pname:pname}
reader.dataDir = pname
reader.JSONfname = pname+'Data'

#create a space for the data in partsDict
reader.partsDict[pname] = dict()

#define the defaults; this must be run first if you want to change the defaults below
reader.defineDefaults()
reader.defineFilterKeys()

#update a few of the options
reader.options['sizeMult'][pname] = 0.002
reader.options['color'][pname] = [1., 1., 0., 1.]
reader.options['center'] = [0,0,0]

reader.addtodict(reader.partsDict,None,pname,'Coordinates',0,0,vals=coordinates)
reader.addtodict(reader.partsDict,None,pname,'Velocities',0,0,vals=velocities)


reader.shuffle_dict()
reader.swap_dict_names()
reader.createJSON()

shuffling ... 
writing JSON files ...
Gaia
Gaia/GaiaDataOptions.json
